In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

In [2]:
data = pd.read_csv("Steel_industry_data.csv")
print(data.isnull().sum())  # Count of missing values in each column
# data['date'] = pd.to_datetime(data['date'])
# y_train=data['Usage_kWh'].values
data = data.drop(columns=['date'])
X_train=data.values

data.head()


date                                    0
Usage_kWh                               0
Lagging_Current_Reactive.Power_kVarh    0
Leading_Current_Reactive_Power_kVarh    0
CO2(tCO2)                               0
Lagging_Current_Power_Factor            0
Leading_Current_Power_Factor            0
NSM                                     0
WeekStatus                              0
Day_of_week                             0
Load_Type                               0
dtype: int64


,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


In [3]:
print(X_train)

[[3.17 2.95 0.0 ... 'Weekday' 'Monday' 'Light_Load']
 [4.0 4.46 0.0 ... 'Weekday' 'Monday' 'Light_Load']
 [3.24 3.28 0.0 ... 'Weekday' 'Monday' 'Light_Load']
 ...
 [3.78 3.17 0.07 ... 'Weekday' 'Monday' 'Light_Load']
 [3.78 3.06 0.11 ... 'Weekday' 'Monday' 'Light_Load']
 [3.67 3.02 0.07 ... 'Weekday' 'Monday' 'Light_Load']]


In [4]:
scaler = StandardScaler()

# Perform scaling and assign it after casting the columns to float
X_train[:, 1:7] = scaler.fit_transform(X_train[:, 1:7]).astype('float64')


print(X_train)
X_train.shape
# df=pd.DataFrame(X_train)
# df.to_csv('scaled_Steel_industry_data.csv', index=False, header=True)

[[3.17 -0.6185163432975694 -0.5213850478056785 ... 'Weekday' 'Monday'
  'Light_Load']
 [4.0 -0.5259110722465691 -0.5213850478056785 ... 'Weekday' 'Monday'
  'Light_Load']
 [3.24 -0.598278105253311 -0.5213850478056785 ... 'Weekday' 'Monday'
  'Light_Load']
 ...
 [3.78 -0.6050241846013972 -0.5119566216410291 ... 'Weekday' 'Monday'
  'Light_Load']
 [3.78 -0.6117702639494832 -0.5065689495469436 ... 'Weekday' 'Monday'
  'Light_Load']
 [3.67 -0.6142233837124237 -0.5119566216410291 ... 'Weekday' 'Monday'
  'Light_Load']]


(35040, 10)

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[7,8])],remainder='passthrough')
X_train=ct.fit_transform(X_train)
print(X_train)
print(X_train.shape)
# df=pd.DataFrame(X_train)
# df.to_csv('scaled_Steel_industry_data.csv', index=False, header=True)


[[1.0 0.0 0.0 ... 0.5132676163619359 -1.6780152604563736 'Light_Load']
 [1.0 0.0 0.0 ... 0.5132676163619359 -1.6419289107691397 'Light_Load']
 [1.0 0.0 0.0 ... 0.5132676163619359 -1.6058425610819058 'Light_Load']
 ...
 [1.0 0.0 0.0 ... 0.5126109334762843 1.6780152604563736 'Light_Load']
 [1.0 0.0 0.0 ... 0.5119542505906323 1.7141016101436073 'Light_Load']
 [1.0 0.0 0.0 ... 0.5126109334762843 -1.7141016101436073 'Light_Load']]
(35040, 17)


In [6]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X_train[:,-1]=le.fit_transform(X_train[:,-1])
print(X_train)
type(X_train)
df=pd.DataFrame(X_train)
df.to_csv('scaled_Steel_industry_data.csv', index=False, header=True)

[[1.0 0.0 0.0 ... 0.5132676163619359 -1.6780152604563736 0]
 [1.0 0.0 0.0 ... 0.5132676163619359 -1.6419289107691397 0]
 [1.0 0.0 0.0 ... 0.5132676163619359 -1.6058425610819058 0]
 ...
 [1.0 0.0 0.0 ... 0.5126109334762843 1.6780152604563736 0]
 [1.0 0.0 0.0 ... 0.5119542505906323 1.7141016101436073 0]
 [1.0 0.0 0.0 ... 0.5126109334762843 -1.7141016101436073 0]]


In [7]:
# Ensure data_scaled is converted to a NumPy array before splitting
node_data = np.array_split(X_train, 8)


## SVR Implementation


In [8]:
def federated_averaging(models, top_k=5):
    """Aggregate model parameters using the average with shape alignment."""
    
    # Collect all support vectors and dual coefficients
    support_vectors = [model.support_vectors_ for model in models]
    dual_coefs = [model.dual_coef_ for model in models]
    intercepts = [model.intercept_ for model in models]

    # Find the minimum number of support vectors to use (to ensure alignment)
    k_values = [sv.shape[0] for sv in support_vectors if isinstance(sv, np.ndarray)]
    
    # Check if we have any valid support vector counts
    if not k_values:
        raise ValueError("No valid support vectors found in any model.")
    
    k = min(min(k_values), top_k)  # Take the minimum of the counts or top_k

    # Select only the top-k support vectors and corresponding coefficients
    aligned_support_vectors = np.vstack([sv[:k] for sv in support_vectors])
    aligned_dual_coefs = np.vstack([coef[:, :k] for coef in dual_coefs if coef.shape[1] > 0])

    # Compute the average dual coefficients and intercepts
    dual_coefs_avg = np.mean(aligned_dual_coefs, axis=0)
    intercept_avg = np.mean(intercepts)

    return aligned_support_vectors, dual_coefs_avg, intercept_avg
# Function to train the local SVR model on each node's data
def train_local_model(data):
    """Train an SVR model on the local node's data."""
    
    y = data[:, 11]  # Target (Usage_kWh)
    X = np.concatenate((data[:, :11], data[:, 12:]), axis=1)  # Features
    
    # Train the SVR model with RBF kernel
    model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
    model.fit(X, y)
    
    return model

## FL using SVR

In [9]:
# Global SVR model initialization
global_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)

In [10]:
for iteration in range(3):
    print(f"Iteration {iteration + 1}:")

    local_models = []

    # Train local models on each node's subset data
    for i, node in enumerate(node_data):
        start = iteration * 1460
        end = start + 1460
        node_subset = node[start:end]

        # Train the local SVR model
        local_model = train_local_model(node_subset)
        local_models.append(local_model)

        # Evaluate the local model
        X_eval = np.concatenate((node_subset[:, :11], node_subset[:, 12:]), axis=1)
        y_eval = node_subset[:, 11]
        y_pred = local_model.predict(X_eval)
        mse = mean_squared_error(y_eval, y_pred)
        print(f"  Node {i + 1} - MSE: {mse:.4f}")

    # Aggregate local models using federated averaging
    support_vectors, dual_coefs_avg, intercept_avg = federated_averaging(local_models)
    if dual_coefs_avg.ndim > 1:
        dual_coefs_avg = dual_coefs_avg.flatten()

    # Update the global model with aggregated parameters
    global_model.fit(support_vectors, dual_coefs_avg)  # Fit on aggregated support vectors
    global_model.intercept_ = intercept_avg  # Set the aggregated intercept

    print(f"Global model updated after iteration {iteration + 1}.\n")

Iteration 1:
  Node 1 - MSE: 0.3363
  Node 2 - MSE: 0.1602
  Node 3 - MSE: 0.1396
  Node 4 - MSE: 0.1053
  Node 5 - MSE: 0.1872
  Node 6 - MSE: 0.0412
  Node 7 - MSE: 0.0419
  Node 8 - MSE: 0.1017


ValueError: Found input variables with inconsistent numbers of samples: [40, 5]

In [ ]:
# Final Evaluation (Optional): Use some test data for evaluation
test_data = node_data[0][:1460]  # Example: Using first 1460 rows from node 1 as test data
# X_test = test_data[:, 1:]
# y_test = test_data[:, 1]
test_data

y_test = test_data[:, 11]   # Target (Usage_kWh)
    # Select all columns except the 12th column using slicing
X_test= np.concatenate((test_data[:, :11], test_data[:, 12:]), axis=1)

df=pd.DataFrame(y_test)
df.to_csv('test_data.csv', index=False, header=True)

y_pred = global_model.predict(X_test)
final_mse = mean_squared_error(y_test, y_pred)
print(f"Final Global Model MSE on Test Data: {final_mse:.4f}")

In [13]:
output=np.concatenate((y_pred.reshape(-1,1),y_test.reshape(-1,1)),1)
type(output)
df=pd.DataFrame(output)
df.to_csv("output.csv", index=False, header=True)

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(r2)
#0.8983988342396683 for Linear Reg


In [15]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(y_pred,y_test,color='red')

plt.title("FL using Linear reg")
plt.xlabel('y_pred')
plt.ylabel('y_test')
plt.show()

## Without FL

In [30]:

without_FL=X_train

X_withoutFL= np.concatenate((without_FL[:, :11], without_FL[:, 12:]), axis=1)
y_withoutFL = without_FL[:, 11]   # Target (Usage_kWh)


In [31]:
from sklearn.model_selection import train_test_split

X_withoutFL_train, X_withoutFL_test,y_withoutFL_train,y_withoutFL_test = train_test_split(X_withoutFL,y_withoutFL,test_size=0.25,random_state=0)

In [ ]:
X_withoutFL_test.shape

In [ ]:
X_withoutFL_train.shape

In [ ]:
y_withoutFL_test.shape

In [ ]:
y_withoutFL_train.shape

In [36]:
#SVR
SVRregressor = SVR(kernel='rbf')

In [ ]:
# regressor.fit(X_withoutFL_train,y_withoutFL_train)
SVRregressor.fit(X_withoutFL_train,y_withoutFL_train)

In [ ]:
pred=SVRregressor.predict(X_withoutFL_test)

print(pred)

In [ ]:
plt.scatter(pred,y_withoutFL_test,color='red')

plt.title("Training the same dataset without FL (SVR)")
plt.xlabel('predicted')
plt.ylabel('actual')
plt.show()

## without FL


In [ ]:
plt.scatter(pred,y_withoutFL_test,color='red')

plt.title("Training the same dataset without FL (regular LR)")
plt.xlabel('predicted')
plt.ylabel('actual')
plt.show()

In [ ]:
pred.shape

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_withoutFL_test, pred)
print(r2)